<a href="https://colab.research.google.com/github/harperd/machine-learning/blob/master/notebooks/multiclass-logistic-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiclass Logistic Regression

Use logistic regression to recognize hand-written digits (0 to 9).

## Imports

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import scipy.optimize as opt
import google.colab as colab
import io

from scipy.io import loadmat

# Allow saving our graphs in the notebook
%matplotlib inline

Chart Dark Theme For Google Colab Dark Mode

In [0]:
style.use('dark_background')

## Read Sample Data

In [23]:
mat_file = colab.files.upload()
!ls -l

total 14676
-rw-r--r-- 1 root root 7511764 Jul 29 00:26 'ex3data1 (1).mat'
-rw-r--r-- 1 root root 7511764 Jul 29 00:24  ex3data1.mat
drwxr-xr-x 1 root root    4096 Jul 19 16:14  sample_data


In [22]:
mat_data = loadmat('ex3data1.mat')
mat_data

{'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [30]:
print(f'X Shape: {mat_data["X"].shape}')
print(f'y Shape: {mat_data["y"].shape}')

X Shape: (5000, 400)
y Shape: (5000, 1)
